<center>

**Vježba 6, korak 2 - Vremenski skup podataka i PyTorch LSTM arhitekture**

</center>

U ovo koraku učitavamo spremljeni model i radimo testiranje s istim podacima kao i u prvom koraku. Bitno je da arhitektura neuronske mreže bude ista kao i kod prethodnog koraka - što se postiže klasom ```AirlineLSTM```.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

train_set1 = pd.read_csv('airline-passengers.csv')
train_set2 = train_set1.iloc[:,1:2].values

plt.plot(train_set2, label = 'Airline Passengers Number')
plt.show()

def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(train_set2)

seq_length = 4
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

class AirlineLSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(AirlineLSTM, self).__init__()        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))        
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        _, (h_out, _) = self.lstm(x, (h0, c0))        
        h_out = h_out.view(-1, self.hidden_size)        
        out = self.fc(h_out)        
        return out

Definiramo klasifikator i učitamo hipeparametre iz datoteke (modela).

In [ ]:
input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1

lstm = AirlineLSTM(num_classes, input_size, hidden_size, num_layers)
lstm.load_state_dict(torch.load('airline.pth'))

Testiranje i prikaz vremenske serije. Rezultati bi trebali biti isti kao i u koraku 1.

In [ ]:
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.title('Prediction')
plt.show()